In [1]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
import pandas

In [2]:
scraped_courses_df = pandas.read_csv("UCalgary_MechEng_Core_and_Elective_(Year2-4)_Courses.csv")
scraped_courses = list(scraped_courses_df["Course Number"])
len(scraped_courses)

67

In [3]:
scraped_courses

['Chemistry 379',
 'Digital Engineering 319',
 'Digital Engineering 407',
 'Engineering 311',
 'Engineering 349',
 'Manufacturing Engineering 417',
 'Mechanical Engineering 101',
 'Mechanical Engineering 317',
 'Mechanical Engineering 337',
 'Mechanical Engineering 339',
 'Mechanical Engineering 341',
 'Mechanical Engineering 421',
 'Mechanical Engineering 461',
 'Mechanical Engineering 471',
 'Mechanical Engineering 473',
 'Mechanical Engineering 479',
 'Mechanical Engineering 485',
 'Mechanical Engineering 493',
 'Mechanical Engineering 495',
 'Mechanical Engineering 585',
 'Mechanical Engineering 599',
 'Engineering 501',
 'Engineering 502',
 'Mechanical Engineering 502',
 'Engineering 503',
 'Engineering 504',
 'Mechanical Engineering 501',
 'Mathematics 375',
 'Physics 365',
 'Physics 369',
 'Biomedical Engineering 509',
 'Biomedical Engineering 515',
 'Biomedical Engineering 523',
 'Biomedical Engineering 525',
 'Energy and Environment, Engineering 575',
 'Engineering 515',
 'Mec

In [4]:
url = "https://ucalgary.ca/pubs/calendar/current/en-4-8.html"

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

# Aerospace minor

## 1. Collect core and elective course link texts for webdriver to click on

In [25]:
driver.get(url)

In [26]:
page_soup = soup(driver.page_source, 'lxml')

In [27]:
aerospace_core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl02_ctl02_cnBody"}).find("ul")

In [28]:
core_link_texts = aerospace_core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Aerospace Engineering 410',
 '411',
 'Electrical Engineering 301',
 'Engineering 501',
 '502',
 'Geomatics Engineering 333',
 'Aerospace Engineering Technical Electives ']

In [29]:
import re
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Aerospace Engineering 410',
 '411',
 'Electrical Engineering 301',
 'Engineering 501',
 '502',
 'Geomatics Engineering 333']

In [30]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")

In [31]:
page_soup = soup(driver.page_source, 'lxml')

In [32]:
aero_electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl02_ctl02_cnBody"}).find("ul")

In [33]:
elec_link_texts = aero_electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Computer Engineering 511',
 'Electrical Engineering 519',
 '503',
 '541',
 '569',
 '574',
 'Software Engineering 401',
 '550',
 'Software Engineering for Engineers 544',
 'Electrical Engineering 441',
 '519',
 'Geomatics Engineering 585',
 '589',
 '615',
 '623',
 '625',
 'Software Engineering 401',
 'Mechanical Engineering 570',
 '572',
 '574',
 '595',
 '597',
 'Software Engineering 401',
 'Mechanical Engineering 521',
 '547',
 '571',
 '573',
 'Software Engineering 401']

In [34]:
elec_link_texts = [link_text for link_text in elec_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
elec_link_texts

['Computer Engineering 511',
 'Electrical Engineering 519',
 '503',
 '541',
 '569',
 '574',
 'Software Engineering 401',
 '550',
 'Software Engineering for Engineers 544',
 'Electrical Engineering 441',
 '519',
 'Geomatics Engineering 585',
 '589',
 '615',
 '623',
 '625',
 'Software Engineering 401',
 'Mechanical Engineering 570',
 '572',
 '574',
 '595',
 '597',
 'Software Engineering 401',
 'Mechanical Engineering 521',
 '547',
 '571',
 '573',
 'Software Engineering 401']

## 2. Scrape core courses

In [37]:
link_texts = core_link_texts

In [39]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            continue
        
        visited_ids.append(table_id)
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        
        course_info = container.findAll("span", {"class": "course-code"})
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text)
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Aerospace Engineering 410
Scraped  Aerospace Engineering 411
Scraped  Software Engineering for Engineers 411
Scraped  Electrical Engineering 301
Scraped  Engineering 501
Scraped  Engineering 502
Scraped  Mechanical Engineering 502
Scraped  Geomatics Engineering 333
Finished scraping 8 courses


## 3. Clean and inspect

In [40]:
false_courses = ["Software Engineering for Engineers 411", "Mechanical Engineering 502"]
core_course_codes_final = []
core_course_names_final = []
core_course_descs_final = []

for i in range(len(core_course_codes)):
    if core_course_codes[i] not in false_courses:
        core_course_codes_final.append(core_course_codes[i])
        core_course_names_final.append(core_course_names[i])
        core_course_descs_final.append(core_course_descs[i])

In [41]:
core_course_codes_final

['Aerospace Engineering 410',
 'Aerospace Engineering 411',
 'Electrical Engineering 301',
 'Engineering 501',
 'Engineering 502',
 'Geomatics Engineering 333']

In [42]:
core_course_names_final

['Aerospace Analysis and Design',
 'Avionics and System Design',
 'Instrumentation, Sensors and Interfacing',
 'Senior Capstone Design Project I',
 'Senior Capstone Design Project II',
 'Computing for Geomatics Engineers']

In [43]:
core_course_descs_final

['Basics of aircraft performance. Topics include: equations of motion; forms of drag; thrust and power; aircraft configuration; weight estimates; and sizing. Teams will design, build and fly a remotely piloted aircraft designed to achieve a certain mission requirement. Students will present their designs with formal design reports and flight demonstrations. ',
 'Introduction to avionics (aviation electronics) in modern aerospace vehicle systems. Impact on vehicle design and performance. Topics include: basic concepts of communication, navigation and surveillance (CNS); radio navigation systems; inertial navigation systems; principles of radar systems; and major communication networks. Students will attain a broad understanding of flight instrumentation, computer control of aircraft, flight control algorithms and system-level analysis. Issues of human-computer interaction will also be discussed. ',
 'An introduction to essential elements of instrumentation and sensing technology. Topics

## 4. Scrape electives courses

In [45]:
link_texts = elec_link_texts

In [62]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html#44840")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html#44840")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        if bool(re.match("[0-9]{3}", link_text)) and course_info[0].text.strip() not in elec_course_codes[-1]:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html#44840")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text)
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html#44840")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Computer Engineering 511
Scraped  Electrical Engineering 519
Scraped  Electrical Engineering 503
503 incorrect subject
Scraped  Electrical Engineering 541
Scraped  Electrical Engineering 569
Scraped  Electrical Engineering 574
574 incorrect subject
Scraped  Software Engineering 401
Software Engineering 401 already scraped
Software Engineering 401 already scraped
Software Engineering 401 already scraped
Scraped  Software Engineering 550
Scraped  Software Engineering for Engineers 544
Scraped  Electrical Engineering 441
519 already scraped
519 incorrect subject
519 incorrect subject
Scraped  Geomatics Engineering 585
Scraped  Geomatics Engineering 589
Scraped  Geomatics Engineering 615
Scraped  Geomatics Engineering 623
Scraped  Geomatics Engineering 625
Software Engineering 401 already scraped
Software Engineering 401 already scraped
Software Engineering 401 already scraped
Software Engineering 401 already scraped
Scraped  Mechanical Engineering 570
Scraped  Mechanical Engineer

In [63]:
elec_course_codes

['Computer Engineering 511',
 'Electrical Engineering 519',
 'Electrical Engineering 503',
 'Electrical Engineering 541',
 'Electrical Engineering 569',
 'Electrical Engineering 574',
 'Software Engineering 401',
 'Software Engineering 550',
 'Software Engineering for Engineers 544',
 'Electrical Engineering 441',
 'Geomatics Engineering 585',
 'Geomatics Engineering 589',
 'Geomatics Engineering 615',
 'Geomatics Engineering 623',
 'Geomatics Engineering 625',
 'Mechanical Engineering 570',
 'Mechanical Engineering 572',
 'Mechanical Engineering 574',
 'Mechanical Engineering 595',
 'Mechanical Engineering 597',
 'Mechanical Engineering 521',
 'Mechanical Engineering 547',
 'Mechanical Engineering 571',
 'Mechanical Engineering 573']

In [64]:
elec_course_names

['Embedded System Interfacing',
 'Special Topics in Electrical Engineering',
 'Computer Vision',
 'Control Systems II',
 'Electronic Systems and Applications',
 'Microwave Engineering',
 'Software Architecture',
 'Engineering Large Scale Analytics Systems',
 'Data Science for Software Engineers',
 'Control Systems I',
 'Wireless Location',
 '3D Visual Perception',
 'Advanced Physical Geodesy',
 'Inertial Surveying and INS/GPS Integration',
 'Advanced GNSS Theory and Applications',
 'Aerodynamics',
 'Computational Fluid Dynamics',
 'Propulsion',
 'Gas Dynamics',
 'Turbomachinery',
 'Materials II',
 'Finite Element Method',
 'Aeroelasticity',
 'Design and Manufacturing of Lightweight Structures']

In [65]:
elec_course_descs

['Review of computer architecture; microcontrollers and their instruction sets; interfacing using common input/output devices, debugging and other software engineering practices, strategies for interrupt handling and exception handling; Interfacing using high level and assembly languages; software and hardware optimizations to achieve real time operations; real time operating systems; Embedded real-time applications.',
 'Current topics in electrical engineering.',
 'Introduction to the fundamentals of image processing and computer vision. Image/video acquisition and raw data matrix manipulation; image processing operations and compression methods; object detection, isolation, and classification; 3D tracking and ego-motion with projective transformations.',
 'Introduction to sampled-data control systems, discretization of analog systems, discrete-time signals and systems, causality, time-invariance, z-transforms, stability, asymptotic tracking, state-space models, controllability and ob

## 5. Combine scraped data and write to CSV

In [66]:
course_codes = core_course_codes_final + elec_course_codes

In [67]:
course_names = core_course_names_final + elec_course_names

In [68]:
course_descs = core_course_descs_final + elec_course_descs

In [70]:
course_descs = [desc.strip() for desc in course_descs]

In [71]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Aerospace Engineering 410,Aerospace Analysis and Design,Basics of aircraft performance. Topics include...
1,Aerospace Engineering 411,Avionics and System Design,Introduction to avionics (aviation electronics...
2,Electrical Engineering 301,"Instrumentation, Sensors and Interfacing",An introduction to essential elements of instr...
3,Engineering 501,Senior Capstone Design Project I,A team-based design course in which students a...
4,Engineering 502,Senior Capstone Design Project II,"A continuation of the capstone design project,..."
5,Geomatics Engineering 333,Computing for Geomatics Engineers,Review of procedural programming and introduct...
6,Computer Engineering 511,Embedded System Interfacing,Review of computer architecture; microcontroll...
7,Electrical Engineering 519,Special Topics in Electrical Engineering,Current topics in electrical engineering.
8,Electrical Engineering 503,Computer Vision,Introduction to the fundamentals of image proc...
9,Electrical Engineering 541,Control Systems II,"Introduction to sampled-data control systems, ..."


In [72]:
df.to_csv('UCalgary_MechEng_Aerospace_minor_Courses.csv', index = False)

# Biomedical

## 1. Collect core course link texts for webdriver to click on

In [73]:
driver.get(url)

In [74]:
page_soup = soup(driver.page_source, 'lxml')

In [75]:
biomed_core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl03_ctl02_cnBody"}).find("ul")

In [76]:
core_link_texts = biomed_core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Biomedical Engineering 301',
 '309',
 '401',
 '415',
 'Biomedical Engineering Technical Electives ',
 'Mechanical Engineering Technical Electives ']

In [77]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Biomedical Engineering 301', '309', '401', '415']

## 2. Scrape core courses

In [79]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        if bool(re.match("[0-9]{3}", link_text)) and course_info[0].text.strip() not in core_course_codes[-1]:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text)
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Biomedical Engineering 301
Scraped  Biomedical Engineering 309
Scraped  Biomedical Engineering 401
Scraped  Biomedical Engineering 415
Finished scraping 4 courses


In [80]:
core_course_codes

['Biomedical Engineering 301',
 'Biomedical Engineering 309',
 'Biomedical Engineering 401',
 'Biomedical Engineering 415']

In [81]:
core_course_names

['Introduction to Biomedical Engineering',
 'Anatomy and Physiology for Engineers',
 'Fundamentals of Biomedical Device and Technologies',
 'Sensor Systems and Data Analytics']

In [82]:
core_course_descs

['Fundamentals of biological systems and the application of engineering principles to address challenges in human and animal health. The role of the biomedical engineer in society and in the current industrial landscape. Topics include pharmaceuticals and drug delivery, instrumentation and devices, physiological and biological measurements, biomechanics, imaging and diagnostics, the Canadian health-care system. Applications may include the cardiovascular, neural and musculo-skeletal systems.',
 'Physiological terminology and anatomical planes of reference; cell biology and physiology; includes structure and function of musculoskeletal, cardiovascular, nervous, gastrointestinal and respiratory tissues and systems; diseases and disorders of those systems; design constraints for bioengineering products.',
 'An introduction to the development of biomedical devices and technologies. Topics may include identifying biomedical needs, concept generation and prototyping, biologically inspired de

## 3. Collect elective course link texts

In [83]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")

In [84]:
page_soup = soup(driver.page_source, 'lxml')

In [85]:
biomed_electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl03_ctl02_cnBody"}).find("ul")

In [86]:
elec_link_texts = biomed_electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Biomedical Engineering 509',
 '511',
 '515',
 '519',
 '523',
 '525',
 '585',
 'Chemical Engineering 535',
 '539',
 'Computer Engineering 509',
 '511',
 'Electrical Engineering 569',
 '606',
 'Engineering 523',
 'Manufacturing Engineering 529']

## 4. Scrape elective courses

In [95]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Biomedical Engineering 509
Scraped  Biomedical Engineering 511
511 incorrect subject
511 incorrect subject
Scraped  Biomedical Engineering 515
519 incorrect subject
Scraped  Biomedical Engineering 519
519 incorrect subject
Scraped  Biomedical Engineering 523
Scraped  Biomedical Engineering 525
525 incorrect subject
Scraped  Biomedical Engineering 585
585 incorrect subject
585 incorrect subject
Scraped  Chemical Engineering 535
Scraped  Chemical Engineering 539
Scraped  Computer Engineering 509
Computer Engineering 509 already scraped
511 already scraped
Scraped  Computer Engineering 511
511 already scraped
Scraped  Electrical Engineering 569
Scraped  Electrical Engineering 606
Scraped  Engineering 523
Engineering 523 already scraped
Scraped  Manufacturing Engineering 529
Finished scraping 15 courses


In [96]:
elec_course_codes

['Biomedical Engineering 509',
 'Biomedical Engineering 511',
 'Biomedical Engineering 515',
 'Biomedical Engineering 519',
 'Biomedical Engineering 523',
 'Biomedical Engineering 525',
 'Biomedical Engineering 585',
 'Chemical Engineering 535',
 'Chemical Engineering 539',
 'Computer Engineering 509',
 'Computer Engineering 511',
 'Electrical Engineering 569',
 'Electrical Engineering 606',
 'Engineering 523',
 'Manufacturing Engineering 529']

In [97]:
elec_course_names

['Introduction to Biomedical Imaging and Applications',
 'Biomaterials and Biocompatibility',
 'Bioengineering Methods in Systems Biology and Physiology',
 'Special Topics in Biomedical Engineering',
 'Biomechanics of Movement',
 'Biomechanics of Tissues',
 'Molecular, Cellular and Tissue Engineering',
 'Principles of Biochemical Engineering',
 'Polymer Engineering',
 'Fundamentals of Biometric Systems Design',
 'Embedded System Interfacing',
 'Electronic Systems and Applications',
 'Optical Instrumentation',
 'Bio-inspired Design',
 'Introduction to Microelectromechanical Systems']

In [98]:
elec_course_descs

['Principles of various imaging modalities used in Biomedical engineering applications, including CT, MRI, ultrasound, PET, SPECT. Image processing operations: filtering, enhancement, feature extraction, pattern recognition and image reconstruction. Image registration and integration of different imaging modalities.',
 'Basic chemical and mechanical properties of biological and synthetic materials and their role in biological system health, dysfunction, and repair. Role of microstructure, material properties, and biocompatibility aspects in selection of biomaterials for medical or industrial applications. Incorporation of biomimetic concepts in material design. Topics may include artificial and tissue engineered products, implants, prostheses, biofilms, biosensors, and foreign body response.',
 'Concepts from systems theory, differential equations, and stochastic processes applied to physiological and biological systems. Experimental and computational approaches to the study of gene ex

## 5. Combine scraped data and write to CSV

In [101]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [102]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Biomedical Engineering 301,Introduction to Biomedical Engineering,Fundamentals of biological systems and the app...
1,Biomedical Engineering 309,Anatomy and Physiology for Engineers,Physiological terminology and anatomical plane...
2,Biomedical Engineering 401,Fundamentals of Biomedical Device and Technolo...,An introduction to the development of biomedic...
3,Biomedical Engineering 415,Sensor Systems and Data Analytics,Introduction to matrix and tensor manipulation...
4,Biomedical Engineering 509,Introduction to Biomedical Imaging and Applica...,Principles of various imaging modalities used ...
5,Biomedical Engineering 511,Biomaterials and Biocompatibility,Basic chemical and mechanical properties of bi...
6,Biomedical Engineering 515,Bioengineering Methods in Systems Biology and ...,"Concepts from systems theory, differential equ..."
7,Biomedical Engineering 519,Special Topics in Biomedical Engineering,Current topics in Biomedical Engineering.
8,Biomedical Engineering 523,Biomechanics of Movement,"Introduction to musculoskeletal biomechanics, ..."
9,Biomedical Engineering 525,Biomechanics of Tissues,The structure and functional behaviour of comp...


In [103]:
df.to_csv('UCalgary_MechEng_Biomedical_minor_Courses.csv', index = False)

# Digital

## 1. Collect core course link texts for webdriver to click on

In [104]:
driver.get(url)

In [105]:
page_soup = soup(driver.page_source, 'lxml')

In [106]:
digital_core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl04_ctl02_cnBody"}).find("ul")

In [107]:
core_link_texts = digital_core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Software Engineering for Engineers 310',
 '311',
 '410',
 '411',
 '510',
 '511',
 'Digital Engineering Technical Electives']

In [108]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Software Engineering for Engineers 310', '311', '410', '411', '510', '511']

## 2. Scrape core courses

In [113]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        if bool(re.match("[0-9]{3}", link_text)) and course_info[0].text.strip() not in core_course_codes[-1]:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Software Engineering for Engineers 310
Scraped  Software Engineering for Engineers 311
Scraped  Software Engineering for Engineers 410
411 incorrect subject
Scraped  Software Engineering for Engineers 411
Scraped  Software Engineering for Engineers 510
Scraped  Software Engineering for Engineers 511
Finished scraping 6 courses


In [114]:
core_course_codes

['Software Engineering for Engineers 310',
 'Software Engineering for Engineers 311',
 'Software Engineering for Engineers 410',
 'Software Engineering for Engineers 411',
 'Software Engineering for Engineers 510',
 'Software Engineering for Engineers 511']

In [115]:
core_course_names

['Fundamentals of Software Design and Development',
 'Advanced Software Design and Development',
 'Fundamentals of Applied Artificial Intelligence',
 'Advanced Applied Artificial Intelligence and Machine Learning',
 'Cyber-Physical Systems',
 'Industrial Internet of Things Systems and Data Analytics']

In [116]:
core_course_descs

['Introduction to design and implementation of software systems for engineering applications. Software design lifecycle. Source code management systems. Debugging and testing techniques. Illustration of common data structures and fundamental algorithms using a high-level language. Libraries for input/output. Software tools to facilitate data analysis and visualization.',
 'Principles of software modeling and design. Key elements of object- oriented design. Advanced topics such as concurrent programming, socket programming, event-driven programming, and database programming. Systems integration techniques to build practical applications from engineering domains.',
 'Techniques for extracting, cleaning, and visualizing data from engineering applications. Basic numerical computation techniques underlying learning algorithms. Fundamental supervised and unsupervised learning algorithms. Emphasis will be on designing practical applications that leverage existing software libraries and framew

## 3. Collect elective course link texts

In [117]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")

In [118]:
page_soup = soup(driver.page_source, 'lxml')

In [119]:
digital_electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl04_ctl02_cnBody"}).find("ul")

In [120]:
elec_link_texts = digital_electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Chemical Engineering 537',
 '561',
 '565',
 'Electrical Engineering 503',
 '573',
 '592',
 'Engineering 515',
 '525',
 'Geomatics Engineering 531',
 '551',
 '559',
 '563',
 '573',
 '585']

## 4. Scrape elective courses

In [121]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Chemical Engineering 537
Scraped  Chemical Engineering 561
561 incorrect subject
Scraped  Chemical Engineering 565
Scraped  Electrical Engineering 503
573 incorrect subject
Scraped  Electrical Engineering 573
573 incorrect subject
573 incorrect subject
Scraped  Electrical Engineering 592
Scraped  Engineering 515
525 incorrect subject
Scraped  Engineering 525
Scraped  Geomatics Engineering 531
Scraped  Geomatics Engineering 551
Scraped  Geomatics Engineering 559
559 already scraped
Scraped  Geomatics Engineering 563
573 incorrect subject
573 already scraped
Scraped  Geomatics Engineering 573
573 already scraped
585 incorrect subject
Scraped  Geomatics Engineering 585
585 incorrect subject
Finished scraping 14 courses


In [122]:
elec_course_codes

['Chemical Engineering 537',
 'Chemical Engineering 561',
 'Chemical Engineering 565',
 'Electrical Engineering 503',
 'Electrical Engineering 573',
 'Electrical Engineering 592',
 'Engineering 515',
 'Engineering 525',
 'Geomatics Engineering 531',
 'Geomatics Engineering 551',
 'Geomatics Engineering 559',
 'Geomatics Engineering 563',
 'Geomatics Engineering 573',
 'Geomatics Engineering 585']

In [123]:
elec_course_names

['Computational Thermodynamics',
 'Machine Learning for Energy Systems',
 'Process Sensors and Data Acquisition',
 'Computer Vision',
 'Computer Networks',
 'Undergraduate Research Thesis I',
 'Project Management for Engineers',
 'Engineering Entrepreneurship',
 'Advanced Photogrammetric and Ranging Techniques',
 'Advanced Geospatial Topics',
 'Digital Imaging and Applications',
 'Data Analysis in Engineering',
 'Digital Terrain Modelling',
 'Wireless Location']

In [124]:
elec_course_descs

['Amalgamation of thermodynamic models and computational techniques with application to industrially important thermodynamic problems such as multi-component flash calculations, reacting systems, phase stability and gas hydrates.',
 'Applications of Machine Learning, Artificial Intelligence and Optimization in Energy Systems. Review of Statistics, Probability and Data Science Concepts; Supervised and Unsupervised Learning in Python including Regression and Clustering; Data Engineering for Data Filtering and Feature Selection; Optimization. Focus on Application to Problems in the Energy Industry.',
 'Introduction to chemical engineering sensors and information processing techniques. Basic concepts of process measurement methods and signals. Data acquisition hardware and software. Identifying errors and uncertainties in process measurements. Filtering and smoothing of process signals. Introduction to Fourier analysis. Devices for measuring temperature, pressure and chemical composition. 

## 5. Combine scraped data and write to CSV

In [125]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [126]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Software Engineering for Engineers 310,Fundamentals of Software Design and Development,Introduction to design and implementation of s...
1,Software Engineering for Engineers 311,Advanced Software Design and Development,Principles of software modeling and design. Ke...
2,Software Engineering for Engineers 410,Fundamentals of Applied Artificial Intelligence,"Techniques for extracting, cleaning, and visua..."
3,Software Engineering for Engineers 411,Advanced Applied Artificial Intelligence and M...,Advanced supervised and unsupervised learning ...
4,Software Engineering for Engineers 510,Cyber-Physical Systems,Fundamental principles related to the design o...
5,Software Engineering for Engineers 511,Industrial Internet of Things Systems and Data...,Fundamentals of Industrial Internet of Things ...
6,Chemical Engineering 537,Computational Thermodynamics,Amalgamation of thermodynamic models and compu...
7,Chemical Engineering 561,Machine Learning for Energy Systems,"Applications of Machine Learning, Artificial I..."
8,Chemical Engineering 565,Process Sensors and Data Acquisition,Introduction to chemical engineering sensors a...
9,Electrical Engineering 503,Computer Vision,Introduction to the fundamentals of image proc...


In [127]:
df.to_csv('UCalgary_MechEng_Digital_minor_Courses.csv', index = False)

# EnergyEnv

## 1. Collect core course link texts for webdriver to click on

In [130]:
driver.get(url)

In [131]:
page_soup = soup(driver.page_source, 'lxml')

In [132]:
core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl05_ctl02_cnBody"}).find("ul")

In [133]:
core_link_texts = core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Energy and Environment, Engineering 355',
 '377',
 'Energy Management 301',
 'Science 529',
 'Energy and Environment Technical Electives ',
 'Mechanical Engineering Technical Electives']

In [134]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Energy and Environment, Engineering 355',
 '377',
 'Energy Management 301',
 'Science 529']

## 2. Scrape core courses

In [135]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        if bool(re.match("[0-9]{3}", link_text)) and course_info[0].text.strip() not in core_course_codes[-1]:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Energy and Environment, Engineering 355
Scraped  Energy and Environment, Engineering 377
Scraped  Energy Management 301
Scraped  Science 529
Finished scraping 4 courses


In [136]:
core_course_codes

['Energy and Environment, Engineering 355',
 'Energy and Environment, Engineering 377',
 'Energy Management 301',
 'Science 529']

In [137]:
core_course_names

['Introduction to Energy and the Environment',
 'Introduction to Environmental Engineering and Sustainability',
 'Electricity Regulation',
 'Project Course in Sustainable Energy, Environment and Economy']

In [138]:
core_course_descs

['History of energy technologies, energetics of natural systems and agriculture, formation, extraction, and transformations of fossil fuels, renewables such as biomass, solar and wind; and the electricity system, environmental impacts of energy systems, technical options for transforming energy systems to reduce environmental impacts.',
 'Theory and practice of environmental engineering. Theory and practice of sustainable development, with a multidisciplinary approach, including components such as engineering, economics, carbon foot-print, policy planning, social license. Comparison of the roles of environmental engineering and sustainable development in society. Environmental and sustainability focused ethics issues including Indigenous engagement, treaty rights and Indigenous rights. Indigenous environmental perspectives. ',
 'An examination of energy development and the business, law and policy issues arising from the development of different electricity sources (nuclear and renewab

## 3. Collect elective course link texts

In [139]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")

In [140]:
page_soup = soup(driver.page_source, 'lxml')

In [141]:
electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl05_ctl02_cnBody"}).find("ul")

In [142]:
elec_link_texts = electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Chemical Engineering 530',
 'Civil Engineering 502',
 '508',
 '581',
 'Computer Engineering 509',
 '511',
 'Electrical Engineering 562',
 '584',
 '585',
 '587',
 '597',
 'Energy and Environment, Engineering 501',
 '503',
 '505',
 '507',
 '519',
 '575',
 '577',
 'Engineering 523',
 'Geomatics Engineering 551',
 '559',
 '573',
 '583',
 'Mechanical Engineering 583',
 '570',
 '595',
 '597',
 'Petroleum Engineering 523',
 '533',
 '555',
 '561',
 '571']

## 4. Scrape elective courses

In [143]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-14.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Chemical Engineering 530
Scraped  Civil Engineering 502
Scraped  Civil Engineering 508
Scraped  Civil Engineering 581
Scraped  Computer Engineering 509
Computer Engineering 509 already scraped
511 incorrect subject
Scraped  Computer Engineering 511
511 already scraped
Scraped  Electrical Engineering 562
Scraped  Electrical Engineering 584
585 incorrect subject
585 incorrect subject
Scraped  Electrical Engineering 585
Scraped  Electrical Engineering 587
597 incorrect subject
Scraped  Electrical Engineering 597
597 incorrect subject
Scraped  Energy and Environment, Engineering 501
503 incorrect subject
Scraped  Energy and Environment, Engineering 503
Scraped  Energy and Environment, Engineering 505
Scraped  Energy and Environment, Engineering 507
519 incorrect subject
519 incorrect subject
Scraped  Energy and Environment, Engineering 519
Scraped  Energy and Environment, Engineering 575
Scraped  Energy and Environment, Engineering 577
Scraped  Engineering 523
Engineering 523 alre

In [144]:
elec_course_codes

['Chemical Engineering 530',
 'Civil Engineering 502',
 'Civil Engineering 508',
 'Civil Engineering 581',
 'Computer Engineering 509',
 'Computer Engineering 511',
 'Electrical Engineering 562',
 'Electrical Engineering 584',
 'Electrical Engineering 585',
 'Electrical Engineering 587',
 'Electrical Engineering 597',
 'Energy and Environment, Engineering 501',
 'Energy and Environment, Engineering 503',
 'Energy and Environment, Engineering 505',
 'Energy and Environment, Engineering 507',
 'Energy and Environment, Engineering 519',
 'Energy and Environment, Engineering 575',
 'Energy and Environment, Engineering 577',
 'Engineering 523',
 'Geomatics Engineering 551',
 'Geomatics Engineering 559',
 'Geomatics Engineering 573',
 'Geomatics Engineering 583',
 'Mechanical Engineering 583',
 'Mechanical Engineering 570',
 'Mechanical Engineering 595',
 'Mechanical Engineering 597',
 'Petroleum Engineering 523',
 'Petroleum Engineering 533',
 'Petroleum Engineering 555',
 'Petroleum Engine

In [145]:
elec_course_names

['Electrochemical Engineering',
 'Civil Engineering Aspects of Sustainable Communities',
 'Environmental Aspects of Energy',
 'Environmental Engineering II',
 'Fundamentals of Biometric Systems Design',
 'Embedded System Interfacing',
 'Photovoltaic Systems Engineering',
 'Electrical Power Systems in Commercial and Institutional Buildings',
 'Introduction to Power Electronics',
 'Power Systems Analysis',
 'Power Systems Operation and Markets',
 'Pollution Prevention and Control for Energy Industry',
 'Life Cycle Assessment',
 'Effluent Treatment Processes for Energy Industry',
 'Introduction to Sustainable Development',
 'Special Topics in Energy and Environment',
 'Alternative Energy Systems',
 'Electrical Transmission System Planning and Operation',
 'Bio-inspired Design',
 'Advanced Geospatial Topics',
 'Digital Imaging and Applications',
 'Digital Terrain Modelling',
 'Environmental Modelling',
 'Mechanical Systems in Buildings',
 'Aerodynamics',
 'Gas Dynamics',
 'Turbomachinery',

In [146]:
elec_course_descs

['Electrochemical kinetics and thermodynamics. Mass transport in electrochemical cells. Design and modelling of electrochemical cells. Application of electrochemistry to fuel cells, batteries, and water treatment.',
 'Definition of sustainability; global urbanization; emissions from transportation systems; economics of urban development from a civil infrastructure point of view, water/wastewater, land use/transportation; public transportation; travel demand management for sustainability; construction industry - energy use and emissions.',
 'Environmental assessment and management in the energy sector. Ecological footprint introduction. Site investigation, field techniques and program implementation; remedial planning and design; cost and time analysis; physical, chemical and biological remediation techniques; biomass and waste to energy; energy use and emissions in transportation systems; energy efficiencies and emissions in building construction; assess problems with energy use from a

## 5. Combine scraped data and write to CSV

In [147]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [148]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,"Energy and Environment, Engineering 355",Introduction to Energy and the Environment,"History of energy technologies, energetics of ..."
1,"Energy and Environment, Engineering 377",Introduction to Environmental Engineering and ...,Theory and practice of environmental engineeri...
2,Energy Management 301,Electricity Regulation,An examination of energy development and the b...
3,Science 529,"Project Course in Sustainable Energy, Environm...",Intended to achieve integration across the int...
4,Chemical Engineering 530,Electrochemical Engineering,Electrochemical kinetics and thermodynamics. M...
5,Civil Engineering 502,Civil Engineering Aspects of Sustainable Commu...,Definition of sustainability; global urbanizat...
6,Civil Engineering 508,Environmental Aspects of Energy,Environmental assessment and management in the...
7,Civil Engineering 581,Environmental Engineering II,"Water and wastewater quantities and quality, w..."
8,Computer Engineering 509,Fundamentals of Biometric Systems Design,"Biometric systems, sensors and devices, Integr..."
9,Computer Engineering 511,Embedded System Interfacing,Review of computer architecture; microcontroll...


In [149]:
df.to_csv('UCalgary_MechEng_EnergyEnv_minor_Courses.csv', index = False)

# Manufacturing

## 1. Collect core course link texts for webdriver to click on

In [150]:
driver.get(url)

In [151]:
page_soup = soup(driver.page_source, 'lxml')

In [153]:
core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl06_ctl02_cnBody"}).find("ul")

In [154]:
core_link_texts = core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Manufacturing Engineering 514',
 'Manufacturing Engineering Technical Electives']

In [155]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Manufacturing Engineering 514']

## 2. Scrape core courses

In [156]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        if bool(re.match("[0-9]{3}", link_text)) and course_info[0].text.strip() not in core_course_codes[-1]:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Manufacturing Engineering 514
Finished scraping 1 courses


In [157]:
core_course_codes

['Manufacturing Engineering 514']

In [158]:
core_course_names

['Integrated Manufacturing Systems']

In [159]:
core_course_descs

['Fundamentals of integrated and competitive manufacturing. Manufacturing and operations strategy. Topics in production and operations management including: production planning and control systems; inventory management systems; process analysis and improvement; quality management systems.']

## 3. Collect elective course link texts

In [160]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")

In [161]:
page_soup = soup(driver.page_source, 'lxml')

In [162]:
electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl11_ctl02_cnBody"}).find("ul")

In [163]:
elec_link_texts = electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Engineering 515',
 'Manufacturing Engineering 501',
 '503',
 '509',
 '517',
 '527',
 '533',
 'Mechanical Engineering 505']

## 4. Scrape elective courses

In [165]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Engineering 515
Engineering 515 already scraped
Scraped  Manufacturing Engineering 501
Manufacturing Engineering 501 already scraped
Scraped  Manufacturing Engineering 503
503 already scraped
Scraped  Manufacturing Engineering 509
509 already scraped
Scraped  Manufacturing Engineering 517
517 already scraped
Scraped  Manufacturing Engineering 527
527 already scraped
Scraped  Manufacturing Engineering 533
533 already scraped
533 already scraped
533 incorrect subject
Scraped  Mechanical Engineering 505
Mechanical Engineering 505 already scraped
Mechanical Engineering 505 already scraped
Finished scraping 8 courses


In [166]:
elec_course_codes

['Engineering 515',
 'Manufacturing Engineering 501',
 'Manufacturing Engineering 503',
 'Manufacturing Engineering 509',
 'Manufacturing Engineering 517',
 'Manufacturing Engineering 527',
 'Manufacturing Engineering 533',
 'Mechanical Engineering 505']

In [167]:
elec_course_names

['Project Management for Engineers',
 'Modelling and Simulation of Manufacturing Systems',
 'Computer-Aided Design and Manufacturing',
 'Advanced Manufacturing Systems',
 'Experimental Design and Analysis',
 'Project Engineering',
 'Computer-Based Control for Industrial Automation',
 'Robotics']

In [168]:
elec_course_descs

['Covers the application of project management principles such as planning, scope development, design, procurement, construction, commissioning and start-up to engineering projects. Class reviews aspects of a current major engineering projects and case studies.',
 'General modelling of production systems. Spreadsheet modelling for capacity analysis. Fundamentals of discrete-event simulation including: key concepts; simulation world views; the simulation study life cycle. Modelling and programming aspects of discrete-event simulation including: verification and validation; simulation animation; interfacing simulation software with other systems. Statistical aspects of discrete-event simulation including: random number and random variate generation; input process modelling; output analysis; variance reduction techniques. Applications of discrete-event simulation to the design and analysis of manufacturing systems.',
 'Hardware and software for computer-aided design and manufacturing (CAD

## 5. Combine scraped data and write to CSV

In [169]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [170]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Manufacturing Engineering 514,Integrated Manufacturing Systems,Fundamentals of integrated and competitive man...
1,Engineering 515,Project Management for Engineers,Covers the application of project management p...
2,Manufacturing Engineering 501,Modelling and Simulation of Manufacturing Systems,General modelling of production systems. Sprea...
3,Manufacturing Engineering 503,Computer-Aided Design and Manufacturing,Hardware and software for computer-aided desig...
4,Manufacturing Engineering 509,Advanced Manufacturing Systems,Manufacturing strategy and competitive manufac...
5,Manufacturing Engineering 517,Experimental Design and Analysis,Introduction to statistical Design of Experime...
6,Manufacturing Engineering 527,Project Engineering,"The project lifecycle. Project planning, sched..."
7,Manufacturing Engineering 533,Computer-Based Control for Industrial Automation,Concepts of digital control. Digital circuits....
8,Mechanical Engineering 505,Robotics,"Kinematics, statics, dynamics and control of r..."


In [171]:
df.to_csv('UCalgary_MechEng_Manufacturing_minor_Courses.csv', index = False)

# Mechatronics

## 1. Collect core course link texts for webdriver to click on

In [172]:
driver.get(url)

In [173]:
page_soup = soup(driver.page_source, 'lxml')

In [174]:
core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl07_ctl02_cnBody"}).find("ul")

In [175]:
core_link_texts = core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Electrical Engineering 301',
 '327',
 'Geomatics Engineering 333',
 'Mechanical Engineering 501',
 '502',
 '561',
 '562',
 'Mechatronics Technical Electives ']

In [176]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Electrical Engineering 301',
 '327',
 'Geomatics Engineering 333',
 'Mechanical Engineering 501',
 '502',
 '561',
 '562']

## 2. Scrape core courses

In [179]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in core_course_codes[-1] or core_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Electrical Engineering 301
Electrical Engineering 301 already scraped
Scraped  Electrical Engineering 327
Scraped  Geomatics Engineering 333
Geomatics Engineering 333 already scraped
Scraped  Mechanical Engineering 501
Mechanical Engineering 501 already scraped
Mechanical Engineering 501 already scraped
502 incorrect subject
Scraped  Mechanical Engineering 502
502 incorrect subject
502 already scraped
502 already scraped
502 incorrect subject
Scraped  Mechanical Engineering 561
561 already scraped
561 incorrect subject
561 incorrect subject
Scraped  Mechanical Engineering 562
Finished scraping 7 courses


In [180]:
core_course_codes

['Electrical Engineering 301',
 'Electrical Engineering 327',
 'Geomatics Engineering 333',
 'Mechanical Engineering 501',
 'Mechanical Engineering 502',
 'Mechanical Engineering 561',
 'Mechanical Engineering 562']

In [181]:
core_course_names

['Instrumentation, Sensors and Interfacing',
 'Signals and Transforms',
 'Computing for Geomatics Engineers',
 'Mechanical Engineering Capstone Design Project I',
 'Mechanical Engineering Capstone Design Project II',
 'Mechatronics Design Laboratory I',
 'Mechatronics Design Laboratory II']

In [182]:
core_course_descs

['An introduction to essential elements of instrumentation and sensing technology. Topics include embedded system programming, basic inputs such as sensors, switches, and keyboards; basic outputs such as motors, relays, LEDs, displays, and speakers; associated circuitry for inputs and outputs; the basics of communications between devices; and power supplies such as linear, switching, and batteries. Topics will be reinforced through weekly hands-on labs.',
 'Continuous-time systems. Impulse response and convolution. Fourier series and Fourier transform. Basics of discrete time signals. Sampling theory. Discrete convolution. Difference equations and the Z-transform. Discrete-time Fourier representations.',
 'Review of procedural programming and introduction to object-based programming using high level compiled and interpreted languages. Binary and ASCII File I/O, use of function libraries and class libraries. Construction of simple classes. Inheritance and polymorphism. Programming for G

## 3. Collect elective course link texts

In [183]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")

In [184]:
page_soup = soup(driver.page_source, 'lxml')

In [185]:
electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl12_ctl02_cnBody"}).find("ul")

In [186]:
elec_link_texts = electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Electrical Engineering 503',
 '525',
 '541',
 '582',
 'Manufacturing Engineering 529',
 '533',
 'Mechanical Engineering 505',
 '547',
 '641',
 '643',
 '650']

## 4. Scrape elective courses

In [187]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Electrical Engineering 503
525 incorrect subject
Scraped  Electrical Engineering 525
Scraped  Electrical Engineering 541
Scraped  Electrical Engineering 582
Scraped  Manufacturing Engineering 529
Scraped  Manufacturing Engineering 533
533 already scraped
533 already scraped
533 incorrect subject
Scraped  Mechanical Engineering 505
Mechanical Engineering 505 already scraped
Mechanical Engineering 505 already scraped
Scraped  Mechanical Engineering 547
547 already scraped
Scraped  Mechanical Engineering 641
Scraped  Mechanical Engineering 643
Scraped  Mechanical Engineering 650
Finished scraping 11 courses


In [188]:
elec_course_codes

['Electrical Engineering 503',
 'Electrical Engineering 525',
 'Electrical Engineering 541',
 'Electrical Engineering 582',
 'Manufacturing Engineering 529',
 'Manufacturing Engineering 533',
 'Mechanical Engineering 505',
 'Mechanical Engineering 547',
 'Mechanical Engineering 641',
 'Mechanical Engineering 643',
 'Mechanical Engineering 650']

In [189]:
elec_course_names

['Computer Vision',
 'Machine Learning for Engineers',
 'Control Systems II',
 'Modelling and Control of Electric Machines and Drives',
 'Introduction to Microelectromechanical Systems',
 'Computer-Based Control for Industrial Automation',
 'Robotics',
 'Finite Element Method',
 'Advanced Control Systems',
 'Optimal and Adaptive Control',
 'Mobile Robotics']

In [190]:
elec_course_descs

['Introduction to the fundamentals of image processing and computer vision. Image/video acquisition and raw data matrix manipulation; image processing operations and compression methods; object detection, isolation, and classification; 3D tracking and ego-motion with projective transformations.',
 'Neural networks: neuron models and network architectures, perceptrons, Widrow-Hoff learning and backpropagation algorithm, associative memory, Hebbian learning, pseudo-inverse learning. Fuzzy systems: basic operations and properties of fuzzy sets; fuzzy rule generation and defuzzification of fuzzy logic; fuzzy neural networks. Applications such as pattern recognition, character recognition, stock market prediction, and control.',
 'Introduction to sampled-data control systems, discretization of analog systems, discrete-time signals and systems, causality, time-invariance, z-transforms, stability, asymptotic tracking, state-space models, controllability and observability, pole assignment, dea

## 5. Combine scraped data and write to CSV

In [191]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [192]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Electrical Engineering 301,"Instrumentation, Sensors and Interfacing",An introduction to essential elements of instr...
1,Electrical Engineering 327,Signals and Transforms,Continuous-time systems. Impulse response and ...
2,Geomatics Engineering 333,Computing for Geomatics Engineers,Review of procedural programming and introduct...
3,Mechanical Engineering 501,Mechanical Engineering Capstone Design Project I,A team-based project course with the focus on ...
4,Mechanical Engineering 502,Mechanical Engineering Capstone Design Project II,"A continuation of the capstone design project,..."
5,Mechanical Engineering 561,Mechatronics Design Laboratory I,A hands-on laboratory experience in the design...
6,Mechanical Engineering 562,Mechatronics Design Laboratory II,More sophisticated and in-depth topics in mech...
7,Electrical Engineering 503,Computer Vision,Introduction to the fundamentals of image proc...
8,Electrical Engineering 525,Machine Learning for Engineers,Neural networks: neuron models and network arc...
9,Electrical Engineering 541,Control Systems II,"Introduction to sampled-data control systems, ..."


In [193]:
df.to_csv('UCalgary_MechEng_Mechatronics_minor_Courses.csv', index = False)

# Petroleum

## 1. Collect core course link texts for webdriver to click on

In [194]:
driver.get(url)

In [195]:
page_soup = soup(driver.page_source, 'lxml')

In [196]:
core_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl08_ctl02_cnBody"}).find("ul")

In [197]:
core_link_texts = core_container.findAll("a", {"class": "link-text"})
core_link_texts = [link_text.text for link_text in core_link_texts]
core_link_texts

['Petroleum Engineering 523', 'Petroleum Engineering Technical Electives ']

In [198]:
core_link_texts = [link_text for link_text in core_link_texts if bool(re.match("[A-Za-z, ]*[0-9]{3}", link_text))]
core_link_texts

['Petroleum Engineering 523']

## 2. Scrape core courses

In [199]:
link_texts = core_link_texts

core_course_codes = []
core_course_names = []
core_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(2)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in core_course_codes[-1] or core_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        core_course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        core_course_names.append(course_info[2].text.strip())
        core_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", core_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Petroleum Engineering 523
Finished scraping 1 courses


In [200]:
core_course_codes

['Petroleum Engineering 523']

In [201]:
core_course_names

['Introduction to Reservoir Engineering']

In [202]:
core_course_descs

["Basic concepts of fluid flow in porous media; important reservoir rock and fluid properties affecting productivity; reserve estimation using volumetric and material balance methods in gas, gas-condensate and oil reservoirs; discussion of different reservoir drive mechanisms; aquifer models; decline analysis; Darcy's Law and single phase flow through porous media. Introduction to well testing, solution of radial diffusivity equation corresponding to infinite-acting and pseudo-steady state flow of slightly compressible fluids and real gases."]

## 3. Collect elective course link texts

In [203]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")

In [204]:
page_soup = soup(driver.page_source, 'lxml')

In [205]:
electives_container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl13_ctl02_cnBody"}).find("ul")

In [206]:
elec_link_texts = electives_container.findAll("a", {"class": "link-text"})
elec_link_texts = [link_text.text for link_text in elec_link_texts]
elec_link_texts

['Geology 377',
 'Mechanical Engineering 595',
 '597',
 'Petroleum Engineering 525',
 '533',
 '555',
 '561',
 '563']

## 4. Scrape electives courses

In [207]:
link_texts = elec_link_texts

elec_course_codes = []
elec_course_names = []
elec_course_descs = []
counter = 0

visited_ids = [] #to keep track of which link has been clicked and avoid clicking on links with the same course code
#ex. multiple links in the page has text "502" - mech 502 and eng 502

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
time.sleep(3)
    
for link_text in link_texts:
    #go to course page
    try:
        links = driver.find_elements_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(2)
    
    for i in range(len(links)):
        
        link = driver.find_elements_by_link_text(link_text)[i]
        link.click()
        time.sleep(3)
    
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #check if table id has already been visited
        if table_id in visited_ids:
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "already scraped")
            continue
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        course_info = container.findAll("span", {"class": "course-code"})
        
        #check if this course belongs is of the correct subject
        subject = course_info[0].text.strip()
        if bool(re.match("[0-9]{3}", link_text)) and (subject not in elec_course_codes[-1] or elec_course_codes[-1].split()[0] not in subject):
            driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
            time.sleep(2)
            print(link_text, "incorrect subject")
            continue
            
        visited_ids.append(table_id)
        
        elec_course_codes.append(subject + " " + course_info[1].text.strip())
        elec_course_names.append(course_info[2].text.strip())
        elec_course_descs.append(container.find("span", {"class": "course-desc"}).text.strip())
        
        print("Scraped ", elec_course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-8.html")
        time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  Geology 377
Scraped  Mechanical Engineering 595
Scraped  Mechanical Engineering 597
597 already scraped
Scraped  Petroleum Engineering 525
533 incorrect subject
533 incorrect subject
533 incorrect subject
Scraped  Petroleum Engineering 533
Scraped  Petroleum Engineering 555
555 already scraped
561 incorrect subject
561 incorrect subject
Scraped  Petroleum Engineering 561
561 already scraped
Scraped  Petroleum Engineering 563
563 already scraped
Finished scraping 8 courses


In [208]:
elec_course_codes

['Geology 377',
 'Mechanical Engineering 595',
 'Mechanical Engineering 597',
 'Petroleum Engineering 525',
 'Petroleum Engineering 533',
 'Petroleum Engineering 555',
 'Petroleum Engineering 561',
 'Petroleum Engineering 563']

In [209]:
elec_course_names

['Petroleum Engineering Geology',
 'Gas Dynamics',
 'Turbomachinery',
 'Waterflooding and Enhanced Oil Recovery',
 'Petroleum Production Engineering',
 'Oil and Gas Field Safety and Equipment',
 'Fuel Science and Technology',
 'Corrosion in Engineering Applications']

In [210]:
elec_course_descs

['The principles and methods of physical geology with special emphasis on their application to the exploitation of oil and gas. Laboratory: properties of minerals and rocks, analysis and interpretation of surface and subsurface maps, interpretation of borehole logs and core, properties of sedimentary rocks.',
 'Fundamentals of one-dimensional gas dynamics. Isentropic and non-isentropic flows, applications of dynamical similarity to shock waves. Oblique shocks, supersonic nozzles, flows with friction or heat transfer. Introduction to computational fluid dynamics (CFD).',
 'Performance of turbomachines, machine selection, Reynolds number and scale effects. Two dimensional flow in turbomachines, degree of reaction and vector diagrams; flow irreversibilities and loss coefficients; pump, compressor and turbine efficiencies. Design of pumps, fans, centrifugal compressors, axial-flow compressors, and axial-flow turbines. Combination of machines with pipes or ducts.',
 'Review of rock-fluid pr

## 5. Combine scraped data and write to CSV

In [211]:
course_codes = core_course_codes + elec_course_codes
course_names = core_course_names + elec_course_names
course_descs = core_course_descs + elec_course_descs

In [212]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,Petroleum Engineering 523,Introduction to Reservoir Engineering,Basic concepts of fluid flow in porous media; ...
1,Geology 377,Petroleum Engineering Geology,The principles and methods of physical geology...
2,Mechanical Engineering 595,Gas Dynamics,Fundamentals of one-dimensional gas dynamics. ...
3,Mechanical Engineering 597,Turbomachinery,"Performance of turbomachines, machine selectio..."
4,Petroleum Engineering 525,Waterflooding and Enhanced Oil Recovery,Review of rock-fluid properties; trapping and ...
5,Petroleum Engineering 533,Petroleum Production Engineering,Principles of oil and gas production mechanics...
6,Petroleum Engineering 555,Oil and Gas Field Safety and Equipment,"Review of safety issues, blow outs, fire and o..."
7,Petroleum Engineering 561,Fuel Science and Technology,"Classification of fuels. Origin, geology, prod..."
8,Petroleum Engineering 563,Corrosion in Engineering Applications,Electrochemical principle of corrosion reactio...


In [213]:
df.to_csv('UCalgary_MechEng_Petroleum_minor_Courses.csv', index = False)

In [214]:
driver.quit()